# 제5장 심층 학습에 기초한 통계적 파라메트릭 음성 합성

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r9y9/ttslearn/blob/master/notebooks/ch05_DNNTTS.ipynb)

## 준비

### Python version

In [ ]:
!python -VV

### ttslearn 설치

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

### 패키지 가져오기

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload
import IPython
from IPython.display import Audio
import os
import numpy as np
import torch
import librosa
import librosa.display

In [ ]:
# 시드 고정
from ttslearn.util import init_seed
init_seed(1234)

### 그래프 그리기 설정 (描画周りの設定) // 번역 수정 필요

In [ ]:
from ttslearn.notebook import get_cmap, init_plot_style, savefig
cmap = get_cmap()
init_plot_style()

## 5.3 풀 콘텍스트 라벨이란?

### 모노폰 라벨

In [ ]:
from nnmnkwii.io import hts
import ttslearn
from os.path import basename

labels = hts.load(ttslearn.util.example_label_file(mono=True))
print(labels[:6])

In [ ]:
# 초 단위로 변환
# NOTE: 100 나노초 단위: 100 * 1e-9 = 1e-7
for s,e,l in labels[:6]:
    print(s*1e-7, e*1e-7, l)

### 풀 콘텍스트 라벨이란?

In [ ]:
labels = hts.load(ttslearn.util.example_label_file(mono=False))
for start_time, end_time, context in labels[:6]:
    print(f"{start_time} {end_time} {context}")

## 5.4 언어 특징량 추출

### Open JTalk로 언어 특징 추출

In [ ]:
import pyopenjtalk

pyopenjtalk.g2p("今日もいい天気ですね", kana=True)

In [ ]:
pyopenjtalk.g2p("今日もいい天気ですね", kana=False)

In [ ]:
labels = pyopenjtalk.extract_fullcontext("今日")
for label in labels:
    print(label)

### HTS 형식의 질문 파일

In [ ]:
qst_path = ttslearn.util.example_qst_file()
! cat $qst_path | grep QS | head -1
! cat $qst_path | grep CQS | head -1

In [ ]:
! head {ttslearn.util.example_qst_file()}

In [ ]:
! tail {ttslearn.util.example_qst_file()}

### HTS 형식 질문 파일 로드

In [ ]:
from nnmnkwii.io import hts
import ttslearn

binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 첫 번째 질문을 확인합니다.
name, ex = binary_dict[0]
print("이진 특징량의 수:", len(binary_dict))
print("수치 특징량의 수:", len(numeric_dict))
print("첫 번째 질문:", name, ex)

### 풀 컨텍스트 라벨을 숫자 표현으로 변환

In [ ]:
from nnmnkwii.frontend import merlin as fe

labels = hts.load(ttslearn.util.example_label_file())
feats = fe.linguistic_features(labels, binary_dict, numeric_dict)
print("언어 특징량(음소 단위)의 사이즈:", feats.shape)

In [ ]:
feats

### 언어 특징량을 프레임 단위로 전개

In [ ]:
feats_phoneme = fe.linguistic_features(labels, binary_dict, numeric_dict, add_frame_features=False)
feats_frame = fe.linguistic_features(labels, binary_dict, numeric_dict, add_frame_features=True)
print("언어 특징량(음소 단위)의 사이즈:", feats_phoneme.shape)
print("언어 특징량(프레임 단위)의 사이즈:", feats_frame.shape)

### 언어 특징의 시각화 (bonus)

In [ ]:
# 시각화를 위해 정규화
in_feats = feats_frame / np.maximum(1, np.abs(feats_frame).max(0))
fig, ax = plt.subplots(figsize=(8,4))
mesh = ax.imshow(in_feats.T, aspect="auto", interpolation="none", origin="lower", cmap=cmap)
fig.colorbar(mesh, ax=ax)

ax.set_xlabel("Time [frame]")
ax.set_ylabel("Context")
plt.tight_layout()

## 5.5 음향 특징량 추출

### 로그 기본 주파수

In [ ]:
from scipy.io import wavfile
import pyworld
from nnmnkwii.preprocessing.f0 import interp1d

# 기본 주파수를 로그 기본 주파수로 변환하는 함수
def f0_to_lf0(f0):
    lf0 = f0.copy()
    nonzero_indices = np.nonzero(f0)
    lf0[nonzero_indices] = np.log(f0[nonzero_indices])
    return lf0

# 오디오 파일 로드
sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = x.astype(np.float64)

# DIO에 의한 기본 주파수 추정
f0, timeaxis = pyworld.dio(x, sr)

# 기본 주파수를 로그 기본 주파수로 변환
lf0 = f0_to_lf0(f0)

# 로그 기본 주파수에 대한 선형 보간
clf0 = interp1d(lf0, kind="linear")

# 시각화
fig, ax = plt.subplots(figsize=(8, 3))
ax.plot(timeaxis, np.exp(lf0), linewidth=2, label="F0")
ax.plot(timeaxis, np.exp(clf0), "--", linewidth=2, label="Continuous F0")
ax.set_xlabel("Time [sec]")
ax.set_xticks(np.arange(0.3, 1.4, 0.2))
ax.set_xlim(0.28, 1.43)
ax.set_ylabel("Frequency [Hz]")
ax.legend()
plt.tight_layout()

# 그림 5-6
savefig("fig/dnntts_cf0")

### 유성/무성 플래그

In [ ]:
# DIO에 의한 기본 주파수 추정
f0, timeaxis = pyworld.dio(x, sr)

# 유성/무성 플래그 계산
vuv = (f0 > 0).astype(np.float32)

hop_length = int(sr * 0.005)
fig, ax = plt.subplots(2, 1, figsize=(8,4))
librosa.display.waveshow(x, sr=sr, x_axis="time", ax=ax[0])
ax[1].plot(timeaxis, vuv)
ax[1].set_ylim(-0.1, 1.1)

ax[0].set_title("Waveform")
ax[1].set_title("V/UV")
ax[0].set_xlabel("Time [sec]")
ax[0].set_ylabel("Amplitude")
ax[1].set_xlabel("Time [sec]")
ax[1].set_ylabel("Binary value")

for a in ax:
    a.set_xlim(0.28, 1.43)
    a.set_xticks(np.arange(0.3, 1.4, 0.2))
plt.tight_layout()

# 그림 5-7
savefig("fig/dnntts_vuv")

### 멜-켑스트럼 (MFCC)

In [ ]:
import pysptk

# DIO로 기본 주파수 추정
f0, timeaxis = pyworld.dio(x, sr)

# CheapTrick에 의한 스펙트럼 포락선(envelope) 추정
# 반환값은 파워 스펙트럼인 것에 주의(진폭이 제곱되고 있다)
spectrogram = pyworld.cheaptrick(x, f0, timeaxis, sr)

# 선형 주파수 축을 멜 주파수 척도로 늘린(stretch) 다음 켑스트럼으로 변환
# alpha는 주파수 축의 늘어난 매개변수를 나타냅니다
alpha = pysptk.util.mcepalpha(sr)
# FFT 길이는 샘플링 주파수가 48kHz인 경우 2048
fftlen = pyworld.get_cheaptrick_fft_size(sr)
# 멜켑스트럼(MFCC)의 차원 수는 mgc_order + 1입니다.
# NOTE: 멜 일반화 캅스트람(Mel-generalized cepstrum)의 이니셜,
# 변수 이름을 mgc로 설정합니다.
mgc_order = 59
mgc = pysptk.sp2mc(spectrogram, mgc_order, alpha)

# 멜켑스트럼(MFCC)에서 원래 스펙트럼 포락(envelope)을 복원
# 스펙트럼의 차원 수는 fftlen // 2 + 1 = 1025
spectrogram_reconstructed = pysptk.mc2sp(mgc, alpha, fftlen)

# 시각화
hop_length = int(sr * 0.005)
fig, ax = plt.subplots(3, 1, figsize=(8,8))
ax[0].set_title("Mel-cepstrum")
ax[1].set_title("Reconstructed spectral envelope from Mel-cepstrum")
ax[2].set_title("Spectral envelope of natural speech")

mesh = librosa.display.specshow(mgc.T, sr=sr, hop_length=hop_length, x_axis="time", cmap=cmap, ax=ax[0])
fig.colorbar(mesh, ax=ax[0])
ax[0].set_yticks(np.arange(mgc_order+2)[::10])

log_sp_reconstructed = librosa.power_to_db(np.abs(spectrogram_reconstructed), ref=np.max)
mesh = librosa.display.specshow(log_sp_reconstructed.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[1])
fig.colorbar(mesh, ax=ax[1], format="%+2.f dB")

log_sp = librosa.power_to_db(np.abs(spectrogram), ref=np.max)
mesh = librosa.display.specshow(log_sp.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[2])
fig.colorbar(mesh, ax=ax[2], format="%+2.f dB")

ax[1].set_ylim(0, 12000)
ax[2].set_ylim(0, 12000)

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_xlim(0.28, 1.43)
    a.set_xticks(np.arange(0.3, 1.4, 0.2))

ax[0].set_ylabel("Mel channel")
ax[1].set_ylabel("Frequency [Hz]")
ax[2].set_ylabel("Frequency [Hz]")

plt.tight_layout()

# 그림 5-8
savefig("fig/dnntts_mcep_reconstructed")

In [ ]:
print("압축률:", spectrogram.shape[1]/mgc.shape[1])

### 대역 비주기성 지표 (상용 로그의 지표)

In [ ]:
# DIO로 기본 주파수 추정
f0, timeaxis = pyworld.dio(x, sr)

# D4C에 의한 비주기성 지표 추정
aperiodicity= pyworld.d4c(x, f0, timeaxis, sr)

# 대역별 비주기성 지표로 압축
bap = pyworld.code_aperiodicity(aperiodicity, sr)

# 시각화
hop_length = int(sr * 0.005)
fig, ax = plt.subplots(2, 1, figsize=(8,6))
mesh = librosa.display.specshow(20*np.log10(aperiodicity).T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="linear", cmap=cmap, ax=ax[0])
ax[0].set_title("Aperiodicity")
fig.colorbar(mesh, ax=ax[0], format="%+2.f dB")

mesh = librosa.display.specshow(bap.T, sr=sr, hop_length=hop_length, x_axis="time", cmap=cmap, ax=ax[1])
fig.colorbar(mesh, ax=ax[1], format="%+2.f dB")
ax[1].set_title("Band-aperiodicity")
for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
    a.set_xlim(0.28, 1.43)
    a.set_xticks(np.arange(0.3, 1.4, 0.2))

ax[1].set_yticks(np.arange(5+1))
ax[1].set_ylabel("Frequency band")
plt.tight_layout()

# 그림 5-9
savefig("fig/dnntts_bap")

In [ ]:
print("압축률:", aperiodicity.shape[1]/bap.shape[1])

### 동적 특징량

In [ ]:
def compute_delta(x, w):
    y = np.zeros_like(x)
    # 특징량의 차원별로 동적 특징 량 계산
    for d in range(x.shape[1]):
        y[:, d] = np.correlate(x[:, d], w, mode="same")
    return y

In [ ]:
import librosa

# 스펙트럼 포락(envelope)의 추정
f0, timeaxis = pyworld.dio(x, sr)
spectrogram = pyworld.cheaptrick(x, f0, timeaxis, sr)

# 파워 스펙트럼을 로그로 변환
spectrogram = librosa.power_to_db(spectrogram, ref=np.max)

# 동적 특징량 계산
delta_window1 = [-0.5, 0.0, 0.5] # 1 次動的特徴量に対する窓
delta_window2 = [1.0, -2.0, 1.0] # 2 次動的特徴量に対する窓

# 1차 동적 특징량
delta = compute_delta(spectrogram, delta_window1)

# 2차 동적 특징량
deltadelta = compute_delta(spectrogram, delta_window2)

# 스펙트럼 포락(envelope)에 대한 동적 특징량을 계산하여 시각화
hop_length = int(sr * 0.005)
fig, ax = plt.subplots(3, 1, figsize=(8,8))
ax[0].set_title("Static features")
ax[1].set_title("Dynamic features (1st order)")
ax[2].set_title("Dynamic features (2nd order)")
mesh = librosa.display.specshow(spectrogram.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[0])
fig.colorbar(mesh, ax=ax[0], format="%+2.f dB")
mesh = librosa.display.specshow(delta.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[1])
fig.colorbar(mesh, ax=ax[1], format="%+2.f dB")
mesh = librosa.display.specshow(deltadelta.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[2])
fig.colorbar(mesh, ax=ax[2], format="%+2.f dB")

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
    a.set_ylim(0, 8000)
    a.set_xlim(0.28, 1.43)
    a.set_xticks(np.arange(0.3, 1.4, 0.2))

plt.tight_layout()

# 그림 5-10
savefig("fig/dnntts_dynamic_features")

### 음향 특징량의 결합

In [ ]:
from nnmnkwii.preprocessing import delta_features

# WORLD로 음성 매개 변수 추정
f0, timeaxis = pyworld.dio(x, sr)
spectrogram = pyworld.cheaptrick(x, f0, timeaxis, sr)
aperiodicity = pyworld.d4c(x, f0, timeaxis, sr)

# 스펙트럼 포락(envelope)을 멜켑스트럼(MFCC)으로 변환
mgc_order = 59
alpha = pysptk.util.mcepalpha(sr)
mgc = pysptk.sp2mc(spectrogram, mgc_order, alpha)

# 유성/무성 플래그 계산
vuv = (f0 > 0).astype(np.float32)

# 연속 로그 기본 주파수 계열
lf0 = interp1d(f0_to_lf0(f0), kind="linear")

# 대역 비주기성 지표 (로그의 정수부)
bap = pyworld.code_aperiodicity(aperiodicity, sr)

# 기본 주파수와 유성/무성 플래그를 2차원 행렬 형태로 둡니다.
lf0 = lf0[:, np.newaxis] if len(lf0.shape) == 1 else lf0
vuv = vuv[:, np.newaxis] if len(vuv.shape) == 1 else vuv

# 동적 특징량을 계산하는 창
windows = [
    [1.0],  # 정적 특징량에 대한 창
    [-0.5, 0.0, 0.5],  # 1차 동적 특징량에 대한 창
    [1.0, -2.0, 1.0],  # 2차 동적 특징량에 대한 창
]

# 정적 특징량과 동적 특징량을 결합한 특징량 계산
mgc = delta_features(mgc, windows)
lf0 = delta_features(lf0, windows)
bap = delta_features(bap, windows)

# 모든 특징 량을 결합한 특징 량 만들기
feats = np.hstack([mgc, lf0, vuv, bap])

print(f"멜켑스트럼(MFCC)의 차원수: {mgc.shape[1]}")
print(f"연속 로그 기본 주파수의 차원 수: {lf0.shape[1]}")
print(f"유성 / 무성 플래그의 차원 수: {vuv.shape[1]}")
print(f"대역 비주기성 지표의 차원 수: {bap.shape[1]}")
print(f"결합된 음향 특징량의 차원 수: {feats.shape[1]}")

## 5.6 음성 파형 합성

In [ ]:
from nnmnkwii.paramgen import mlpg
from IPython.display import Audio
import IPython
from ttslearn.dnntts.multistream import get_windows, split_streams
from ttslearn.dsp import world_spss_params

# 오디오 파일 로드
sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = x.astype(np.float64)

# 음향 특징량 추출 파라미터
mgc_order = 59
alpha = pysptk.util.mcepalpha(sr)
fftlen = pyworld.get_cheaptrick_fft_size(sr)

# 음향 특징량 추출
feats = world_spss_params(x, sr, mgc_order)

# 파라미터 생성에 필요한 특징량의 분산 (Variance)
# 6장에서 설명하지만 실제로는 전체 학습 데이터에 대해 계산
feats_var = np.var(feats, axis=1)

# 결합된 특징량으로부터 각 특징량의 분리
stream_sizes = [(mgc_order+1)*3, 3, 1, pyworld.get_num_aperiodicities(sr)*3]
mgc, lf0, vuv, bap = split_streams(feats, stream_sizes)

start_ind = np.hstack(([0], np.cumsum(stream_sizes)[:-1]))
end_ind = np.cumsum(stream_sizes)

# 파라미터 생성에 필요한 동적 특징량을 계산하는 데 사용되는 창
windows = get_windows(num_window=3)

# 파라미터 생성
mgc = mlpg(mgc, feats_var[start_ind[0]:end_ind[0]], windows)
lf0 = mlpg(lf0, feats_var[start_ind[1]:end_ind[1]], windows)
bap = mlpg(bap, feats_var[start_ind[3]:end_ind[3]], windows)

# 멜켑스트럼(MFCC)에서 스펙트럼 포락(envelope)으로의 변환
spectrogram = pysptk.mc2sp(mgc, alpha, fftlen)

# 연속 로그 기본 주파수에서 기본 주파수로 변환
f0 = lf0.copy()
f0[vuv < 0.5] = 0
f0[np.nonzero(f0)] = np.exp(f0[np.nonzero(f0)])

# 대역 비주기 지표를 비주기 지표로 변환
aperiodicity = pyworld.decode_aperiodicity(bap.astype(np.float64), sr, fftlen)

# WORLD로 음성 파형 합성
y = pyworld.synthesize(
    f0.flatten().astype(np.float64),
    spectrogram.astype(np.float64),
    aperiodicity.astype(np.float64),
    sr
)

# 오디오 플레이어 표시
IPython.display.display(Audio(x.astype(np.float32), rate=sr))
IPython.display.display(Audio(y.astype(np.float32), rate=sr))

# 시각화
fig, ax = plt.subplots(2, 1, figsize=(8,4), sharey=True)
ax[0].set_title("Natural speech")
ax[1].set_title("Reconstructed speech by acoustic features")
librosa.display.waveshow(x.astype(np.float32), sr, ax=ax[0])
librosa.display.waveshow(y.astype(np.float32), sr, ax=ax[1])
for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Amplitude")
plt.tight_layout()

In [ ]:
n_fft =  1024
frame_shift = int(sr * 0.005)
X = librosa.stft(x.astype(np.float32), n_fft=n_fft, win_length=n_fft, hop_length=frame_shift, window="hann")
logX = librosa.amplitude_to_db(np.abs(X), ref=np.max)
Y = librosa.stft(y.astype(np.float32), n_fft=n_fft, win_length=n_fft, hop_length=frame_shift, window="hann")
log_Y = librosa.amplitude_to_db(np.abs(Y), ref=np.max)

fig, ax = plt.subplots(2, 1, figsize=(8, 6))
ax[0].set_title("Natural spectrogram")
ax[1].set_title("Reconstructed spectrogram from acoustic features")

mesh = librosa.display.specshow(logX, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[0])
fig.colorbar(mesh, ax=ax[0], format="%+2.f dB")
mesh = librosa.display.specshow(log_Y, sr=sr, hop_length=hop_length, x_axis="time", y_axis="hz", cmap=cmap, ax=ax[1])
fig.colorbar(mesh, ax=ax[1], format="%+2.f dB")

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
    a.set_ylim(0, 8000)
    a.set_xlim(0.28, 1.43)
    a.set_xticks(np.arange(0.3, 1.4, 0.2))

plt.tight_layout()

# 그림 5-13
savefig("fig/dnntts_waveform_reconstruction")